# 트립 어드바이저 데이터 수집



In [5]:
from selenium import webdriver
import time
from pygrok import Grok
import pandas as pd
from ast import literal_eval 
from tqdm import notebook  # for문 진행상황 확인

In [6]:
# 초기값
data_list = []
error_list = []

driver = webdriver.Chrome("./chromedriver")  

In [7]:
def get_rating(rating_html):    
    if "bubble_50" in rating_html:
        rating = 5
    elif "bubble_40" in rating_html:
        rating = 4
    elif "bubble_30" in rating_html:
        rating = 3
    elif "bubble_20" in rating_html:
        rating = 2
    elif "bubble_10" in rating_html:
        rating = 1
    return rating

def get_date(date_text):
    
    print(date_text)
    try:
        if '어제' in date_text:
            date = '2021/1'
            return date
        else:        
            date_pattern = '%{YEAR:year}년 %{MONTHNUM:month}월'
            pattern = Grok(date_pattern)
            grok = pattern.match(date_text)

            if grok is None:
                date_pattern = '%{MONTHNUM:month}월'
                pattern = Grok(date_pattern)
                grok = pattern.match(date_text)        
                grok['year'] = '2021'

            date = grok['year'] + "/" + grok['month']
            return date    
    except:    
        return None

def get_data(page, url, place, category, driver):

    # 사용자 id 수집
    users = driver.find_elements_by_css_selector('._1r_My98y')  

    # 별점 수집
    ratings = driver.find_elements_by_css_selector('.nf9vGX55')
    
    # 리뷰 수집
    review_titles = driver.find_elements_by_css_selector('.ocfR3SKN')
    review_descs = driver.find_elements_by_css_selector('.IRsGHoPm')
    
    # 날짜 수집
    dates = driver.find_elements_by_css_selector('._2fxQ4TOx')
    
    is_normal = False
    
    if len(users) == len(ratings) == len(review_titles) == len(review_descs) == len(dates):
            is_normal = True
    
    if is_normal:
        
        for index in range(len(users)):
                  
            user_link = users[index].get_attribute('href')        
            user_id = user_link.rsplit('/', 1)[-1]
            
            rating_score = get_rating(ratings[index].get_attribute('innerHTML'))
            
            review = review_titles[index].text + "\n" + review_descs[index].text
            
            date = get_date(dates[index].text)
            
            tag = []
            tag.append(category)
            
            data = {}
            data['user_id'] = user_id
            data['rating'] = rating_score
            data['review'] = review
            data['date'] = date
            data['place'] = place
            data['url'] = url
            data['tag'] = tag
            
            print(data)
            data_list.append(data)
    
    else:
        print("수집 중 오류 발생 user id, ratings, review, dates 개수가 서로 다름")
        
        error_data = {}
        error_data['place'] = place
        error_data['url'] = url
        error_data['review_page'] = page
        
        print(error_data)
        error_list.append(error_data)


In [8]:
def get_data_from_url(url, place, category, driver):
    
    driver.get(url)
    time.sleep(3)
    paginations = driver.find_elements_by_css_selector('.ui_pagination a')
    
    if len(paginations) > 1:
        last_page = int(paginations[-1].get_attribute('innerHTML'))
    else:
        last_page = 1
    print('last_page ' + str(last_page))
    
    page = 1
    print("page 1")
    get_data(page, url, place, category, driver) # 첫 페이지 수집하고 시작

    for page in range(2, last_page + 1):
        print("page " + str(page))
        next_button = driver.find_element_by_css_selector('.ui_button.nav.next.primary')
        next_button.click()
        time.sleep(3)
        get_data(page, url, place, category, driver)

In [10]:
# 파일 읽어오기
places_data = pd.read_excel('./places.xlsx')
categories = places_data['category']
places = places_data['place']
urls = places_data['url']

# range(시작, 끝) 예) range(1000, 1500)
for index in notebook.tqdm(range(len(urls))):
    url = urls[index]
    place = places[index]
    print()
    print(place)
    category = categories[index]
    get_data_from_url(url, place, category, driver)

In [11]:
print(len(data_list))
print(len(error_list))

0
0


In [12]:
data_table = pd.DataFrame(data_list)
data_table      


""


In [13]:
error_table = pd.DataFrame(error_list)
error_table

""


In [9]:
# 엑셀로 저장하기
data_table.to_excel("tripadvisor.xlsx")